## make_MAD_normalized_counts_any_division
The purpose of this notebook is to create a dataframe of MAD-normalized fractional counts for each batch for the 4 large brain division, the 12 subdivisions and the 122 subdivisions. 

The MAD-normalized fractional count is calculated like:


$f_{norm} = \frac{f_{orig}-\mathrm{Median}(f_{orig, ctrl})}{\mathrm{MAD}(f_{orig, ctrl})}$,

where:

$f_{orig}$ is the original fractional count in a single region for a single animal

$\mathrm{Median}(f_{orig, ctrl})$ is the median of all fractional counts in a single region for all control animals in a given batch.

$\mathrm{MAD}(f_{orig, ctrl})$ is the median absolute deviation of all fractional counts in a single region for all control animals in a given batch.


In [1]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import edt
import seaborn as sns
import graphviz

%matplotlib inline

## Data loading and validation

In [2]:
main_file = '../data/animals_eroded_counts_20201125.csv'
pma_segment_props_file = '../data/pma_segment_properties_info'
pma_ontology_json_file = '../data/PMA_ontology.json'
structures_file = '../data/structures_6.22.20.csv' 
pivot_table_file = '../data/pivot_20201125.csv'

In [3]:
# Read in the new dataframe containing cell counts in each PMA brain region
main_df = pd.read_csv(main_file)
main_df

,brain,brain_longname,condition,batch,root,Midbrain,Oculomotor nucleus,Medial terminal nucleus of the accessory optic tract,Lateral terminal nucleus of the accessory optic tract,Dorsal terminal nucleus of the accessory optic tract,...,supraoptic commissures,fasciculus retroflexus,habenular commissure,stria medullaris,nigrostriatal tract,rubrospinal tract,ventral tegmental decussation,crossed tectospinal pathway,direct tectospinal pathway,doral tegmental decussation
0,an011,an011,acquisition_day1,202010_cfos,0,13350,0,34,29,24,...,22,57,0,90,95,508,41,27,0,3
1,an012,an012,acquisition_day1,202010_cfos,0,11361,22,40,14,93,...,120,64,0,79,183,387,36,40,0,4
2,an013,an013,acquisition_day1,202010_cfos,1,12563,11,68,39,5,...,26,26,1,68,71,352,26,29,0,1
3,an014,an014,acquisition_day1,202010_cfos,2,12554,3,28,62,98,...,53,70,0,83,109,197,18,51,0,0
4,an015,an015,acquisition_day1,202010_cfos,2,11431,2,20,8,4,...,38,20,1,73,53,421,23,50,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,an30_crus1_lat,an30_crus1_lat,Vehicle_control_reversal,202002_cfos,3,32942,2,216,98,427,...,88,28,3,47,122,864,92,67,0,0
165,an31_crus1_lat,an31_crus1_lat,Vehicle_control_reversal,202002_cfos,1,33377,9,108,162,324,...,85,14,0,97,110,1249,188,96,0,9
166,an32_crus1_lat,an32_crus1_lat,Vehicle_control_reversal,202002_cfos,7,36458,2,208,167,199,...,114,25,4,249,224,1657,94,157,0,4
167,an33_crus1_lat,an33_crus1_lat,Vehicle_control_reversal,202002_cfos,4,29123,4,59,128,481,...,94,39,3,29,90,978,56,107,0,2


In [4]:
batches = main_df['batch'].unique()
conditions = main_df['condition'].unique()

In [5]:
# Read in the pivot table df
pivot_df = pd.read_csv(pivot_table_file)
pivot_df

,condition,201810_adultacutePC_ymaze_cfos,201904_ymaze_cfos,201908_tpham_ymaze_cfos,202002_cfos,202010_cfos
0,CNO_control_no_reversal,0,8,0,0,0
1,CNO_control_reversal,0,10,7,5,0
2,CNOnCrusILT,0,0,8,10,0
3,CNOnCrusIRT,0,0,9,10,0
4,Vehicle_control_reversal,0,0,0,9,0
5,acquisition_day1,0,0,0,0,10
6,acquisition_day2,0,0,0,0,10
7,cno_injection,0,0,0,3,0
8,crusI_bilateral_reversal,6,0,0,0,0
9,dilutedvector_control_reversal,0,0,0,14,0


In [6]:
with open(pma_ontology_json_file,'r') as infile:
    pma_ontology_dict = json.load(infile)

In [7]:
class Node():
    """ For making the ontology graph"""
    def __init__(self,name):
        self.name = name
        self.parent = None
        self.children = [] # list of Node objects
        self.level = None
    def __repr__(self):
        return str(self.name)

In [8]:
class Graph():
    """ The ontology graph """
    def __init__(self):
        self.nodes = {} # map name to Node object
        self.html_str = ''
        self.level = 0
        self.last_checked_nodename = None
    def make_graph(self,dic):
        name = dic.get('name')
        if name not in self.nodes:
            node = Node(name)
            self.nodes[name] = node
        else:
            node = self.nodes[name]
        if name == 'root':
            node.level = 0
        children = dic.get('children')
        self.level += 1
        for child in children: # child is a dict
            child_name = child.get('name')
            child_node = Node(child_name)
            child_node.parent = node
            child_node.level = self.level 
            self.nodes[child_name] = child_node
            node.children.append(child_node)
            self.make_graph(child)
        self.level -= 1
    def print_branch(self,nodename,level=0,stoplevel=2):
        """ 
        ---PURPOSE---
        Print out the branch of this ontology, 
        starting with a parent node name
        ---INPUT---
        nodename     The parent node you want to start at
        level        An internal variable. Do not modify.
        stoplevel    The number of levels down from the parent 
                     node that you want to print. 
                     Use -1 to print the entire branch out
        """
        if stoplevel == -1:
            pass
        elif level > stoplevel:
            return
        
        print("\t"*level,level,nodename)
        for child in self.nodes[nodename].children:
            self.print_branch(child.name,level+1,stoplevel=stoplevel)
        level-=1
    
    def visualize_graph(self,nodename,stoplevel):
        """ 
        ---PURPOSE---
        Visualize a graph starting at a given node
        and stopping at a certain number of levels down 
        the ontology.
        ---INPUT---
        nodename     The parent node you want to start at
        stoplevel    The number of levels down from the parent 
                     node that you want to print
        """
        graph = graphviz.Digraph()
        self.fill_viz_graph(nodename=nodename,graph=graph,level=0,stoplevel=stoplevel)
        return graph
    
    def fill_viz_graph(self,nodename,graph,level,stoplevel):
        if stoplevel == -1:
            pass
        elif level >= stoplevel:
            return
        graph.node(nodename)
        for child in self.nodes[nodename].children:
            graph.edge(nodename,child.name)
            self.fill_viz_graph(child.name,graph=graph,level=level+1,stoplevel=stoplevel)
        level-=1
        return
    def make_html_ontology(self):
        """ 
        ---PURPOSE---
        Print out the entire ontology
        in html format with regions
        that are included in the dataframe in bold
        ---INPUT---
        """
        # reset html_str
        self.html_str = ''
        self.html_helper(nodename='root')
        return self.html_str

    def html_helper(self,nodename):
        node = self.nodes[nodename]
        level = node.level
        
        if nodename == 'root':
            previous_level = -1
        else:
            previous_node = self.nodes[self.last_checked_nodename]
            previous_level = previous_node.level
        
        if level > previous_level:
            self.html_str += '<ul style="list-style-type: none;">'
        elif level == previous_level:
            # dont need to do anything
            pass
        elif level < previous_level:
            # then we need to end unordered lists
            # the number of which we need to end depends on the difference between
            # current and previous level
            level_diff = previous_level-level
            self.html_str += '</ul>'*level_diff
        # Add the actual text to the html string
        # use boldface if nodename is in the volume and therefore a dataframe column
        if nodename in main_df.columns:
            self.html_str += '<li>' + ' '.join([str(level),f'<b>{nodename}</b>','</li>'])
        else:
            self.html_str += '<li>' + ' '.join([str(level),nodename,'</li>'])
        self.last_checked_nodename = nodename
        for child in self.nodes[nodename].children:
            self.html_helper(child.name,)
        return

In [9]:
# Make a graph containing all brain regions as nodes and parent-child relationships as edges
graph = Graph()
graph.make_graph(pma_ontology_dict)

As always we need our get_progeny() function to get the list of progeny given a region

In [10]:
def get_progeny(dic,parent_structure,progeny_list):
    """ 
    ---PURPOSE---
    Get a list of all progeny of a structure name.
    This is a recursive function which is why progeny_list is an
    argument and is not returned.
    ---INPUT---
    dic                  A dictionary representing the JSON file 
                         which contains the ontology of interest
    parent_structure     The structure
    progeny_list         The list to which this function will 
                         append the progeny structures. 
    """
    name = dic.get('name')
    children = dic.get('children')
    if name == parent_structure:
        for child in children: # child is a dict
            child_name = child.get('name')
            progeny_list.append(child_name)
            get_progeny(child,parent_structure=child_name,progeny_list=progeny_list)
        return
    
    for child in children:
        child_name = child.get('name')
        get_progeny(child,parent_structure=parent_structure,progeny_list=progeny_list)
    return

We also need a function to get total counts from a given region for a given brain. 

In [11]:
def get_total_counts(soi,series,debug=False):
    """ Get the total counts in a region
    and all of its descendent regions 
    from a single dataframe series (single brain)
    
    ---INPUT---
    soi    region name of interest
    seires pandas series representing a row (single brain) from the main dataframe
    ---OUTPUT---
    total  the total number of summed
    """
    
    progeny_list=[]
    get_progeny(pma_ontology_dict,parent_structure=soi,progeny_list=progeny_list)
    total=0 #store counts in this list
    # Add the counts for the region itself, if that region is in the dataframe
    try:
        total+=series[soi]
        if debug:
            print(f"{soi} is in dataframe!")
    except:
        if debug:
            print(f"{soi} not in dataframe")
        pass
    for progen in progeny_list:
        try:
            
            count = int(series[progen])
            if debug:
                print(f"{progen} is in dataframe!")
            total+=count    
        except:
            if debug:
                print(f"{progen} not in dataframe")
            pass
    return total

In [17]:
def get_fractional_counts(soi,series,normalizing_counts,debug=False):
    """ Get the fractional counts in a region
    and all of its descendent regions.
    
    ---INPUT---
    soi                 region name of interest
    seires              pandas series representing a row (single brain) from the main dataframe
    normalizing_counts  The total counts to use as the denominator for calculating the fraction 
    ---OUTPUT---
    The fractional count of the single soi out of the entire Telencephalon, Diencephalon, Mesencephalon
    """
    
    progeny_list=[]
    get_progeny(pma_ontology_dict,parent_structure=soi,progeny_list=progeny_list)
    total=0 #store counts in this list
    # Add the counts for the region itself, if that region is in the dataframe
    try:
        total+=series[soi]
        if debug:
            print(f"{soi} is in dataframe!")
    except:
        if debug:
            print(f"{soi} not in dataframe")
        pass
    for progen in progeny_list:
        try:
            count = int(series[progen])
            if debug:
                print(f"{progen} is in dataframe!")
            total+=count    
        except:
            if debug:
                print(f"{progen} not in dataframe")
            pass
    return total/normalizing_counts

# Helper scripts 

In [18]:
# helper functions for plots below
def get_counts_dict(brain_division_dict,batch,condition,count_type):
    """ 
    ---PURPOSE---
    Given a batch and condition, calculate either the total or fractional
    counts in each region for each animal in this batch/condition combo
    in the main dataframe  
    ---INPUT---
    batch          The "experiment" column of the main dataframe
    condition      The "Condition" column of the main dataframe
    count_type     if 'frac' plots using fractional counts. 
                   if 'total' plots using total counts
    ---OUTPUT---
    counts_dict    dictionary[brain][division] = count
    """
    assert count_type in ['frac','total']
    assert batch in main_df['batch'].unique()
    assert condition in main_df['condition'].unique()
    batch_mask = main_df['batch'] == batch
    condition_mask = main_df['condition'] == condition
    comb_mask = np.logical_and(batch_mask,condition_mask)
    batch_condition_df = main_df[comb_mask]
    n_animals = len(batch_condition_df)
    # make a dictionary where keys are Brain names and values are dictionaries mapping brain division
    # to either fractional or total counts
    counts_dict = {}
    for index,brain_series in batch_condition_df.iterrows():
        brain = brain_series['brain']
        # Figure out sum of all regions in the brain division dict to provide normalization
        normalizing_counts = 0
        for division in brain_division_dict:
            brain_regions = brain_division_dict[division]
            total_counts = sum([get_total_counts(region,brain_series) for region in brain_regions])
            normalizing_counts += total_counts

        counts_dict[brain] = {}
        for division in brain_division_dict:
            subregions = brain_division_dict[division]
            if count_type == 'frac':
                counts = [get_fractional_counts(soi,brain_series,normalizing_counts) for soi in subregions]
            elif count_type == 'total':
                counts = [get_total_counts(soi,brain_series) for soi in subregions]
            counts_dict[brain][division] = sum(counts)
    return counts_dict


def create_division_df(brain_division_dict,batch,condition,count_type='frac'):
    """ 
    ---PURPOSE---
    Given a batch and condition create a pandas DataFrame
    containing columns: Brain, Batch, Condition and The major divisions.
    ---INPUT---
    batch          The "experiment" column of the main dataframe
    condition      The "Condition" column of the main dataframe
    count_type     if 'frac' plots using fractional counts. 
                   if 'total' plots using total counts
    """
    counts_dict = get_counts_dict(brain_division_dict,batch,condition,count_type)

    counts_by_division = []
    for brain in counts_dict:
        division_dict = {'brain':brain}
        for division in counts_dict[brain]:
            division_dict[division] = counts_dict[brain][division]
        counts_by_division.append(division_dict)
    df = pd.DataFrame(counts_by_division)
    df = df.assign(batch=batch,condition=condition)
    return df

In [19]:
def make_all_division_df(brain_division_dict,count_type='total'):
    """ 
    ---PURPOSE---
    Given a brain_division_dict create a pandas DataFrame
    containing columns: Brain, Batch, Condition and the subdivisions
    for all batches and conditions.
    ---INPUT---
    batch          The "experiment" column of the main dataframe
    condition      The "Condition" column of the main dataframe
    count_type     if 'frac' plots using fractional counts. 
                   if 'total' plots using total counts
    """
    df_list = []
    for ii,batch in enumerate(batches):
        # figure out which conditions were used in this batch
        condition_list = pivot_df['condition'][pivot_df[batch]>0]
        for condition in condition_list:
            counts_dict = get_counts_dict(brain_division_dict,batch,condition,count_type)

            counts_by_division = []
            for brain in counts_dict:
                division_dict = {'brain':brain}
                for division in counts_dict[brain]:
                    division_dict[division] = counts_dict[brain][division]
                counts_by_division.append(division_dict)
            df = pd.DataFrame(counts_by_division)
            df = df.assign(batch=batch,condition=condition)
            df_list.append(df)
    cdf = pd.concat(df_list)
    # reorder columns to be brain,batch,condition,fractions...
    cols = cdf.columns.tolist()
    newcols = cols[0:1] + cols[-2:] + cols[1:-2]
    cdf = cdf[newcols]
    return cdf

# First, make the fractional counts dataframe

In [106]:
# Make a brain division dict from the list of regions
bdict_subregions = {
    "Olfactory areas (T)":["Olfactory areas"],
    "Isocortex (T)":["Isocortex",],
    "Cortical subplate (T)":["Cortical subplate",],
    "Hippocampal formation (T)":["Hippocampal formation",],
    "Cerebral nuclei (T)":["Cerebral nuclei",],
    'Thalamus (D)':["Thalamus",],
    'Hypothalamus (D)':['Hypothalamus'],
    'Midbrain, motor related (Mes)':['Midbrain, motor related'],
    'Midbrain, sensory related (Mes)':['Midbrain, sensory related'],
    'Midbrain, behavioral state related (Mes)':['Midbrain, behavioral state related'],
    'Medulla (Met)':['Medulla'],
    'Pons (Met)':['Pons'],
    }

In [107]:
df = make_all_division_df(bdict_subregions,count_type='frac')

# Figure out median fractional counts in each division for each batch, control animals only 


In [108]:
batch_control_conditions_dict = {
    '202010_cfos':[],
    '202002_cfos':['CNO_control_reversal','dilutedvector_control_reversal'],
    '201904_ymaze_cfos':['CNO_control_no_reversal','CNO_control_reversal','homecage_control'],
    '201810_adultacutePC_ymaze_cfos':['vector_control_reversal'],
    '201908_tpham_ymaze_cfos':['CNO_control_reversal']
}

In [109]:
df_list = []
for batch in batches:
#     print(batch)
    control_conditions = batch_control_conditions_dict[batch]
    if len(control_conditions) < 1:
        continue
    print(batch)
    df_batch = df[df['batch'] == batch]
    df_batch_controls = df_batch[df_batch['condition'].isin(control_conditions)]
    medians=df_batch_controls.iloc[:,3:].median()
    mads=df_batch_controls.iloc[:,3:].mad()
    df_batch.iloc[:,3:] = (df_batch.iloc[:,3:]-medians)/mads
    df_list.append(df_batch)
    



201904_ymaze_cfos
201908_tpham_ymaze_cfos
202002_cfos
201810_adultacutePC_ymaze_cfos


In [110]:
cdf = pd.concat(df_list)
cdf

,brain,batch,condition,Olfactory areas (T),Isocortex (T),Cortical subplate (T),Hippocampal formation (T),Cerebral nuclei (T),Thalamus (D),Hypothalamus (D),"Midbrain, motor related (Mes)","Midbrain, sensory related (Mes)","Midbrain, behavioral state related (Mes)",Medulla (Met),Pons (Met)
0,an1,201904_ymaze_cfos,CNO_control_no_reversal,0.670307,0.768355,-0.569974,0.850938,-1.474049,-1.030254,-1.402285,-0.126142,0.395868,-0.371397,1.124952,0.408338
1,an2,201904_ymaze_cfos,CNO_control_no_reversal,1.788574,-1.094034,2.450398,0.956282,-0.518060,-1.214898,1.143084,-0.719854,0.652739,0.314057,1.699262,1.347025
2,an3,201904_ymaze_cfos,CNO_control_no_reversal,1.939475,1.980944,-0.202607,-0.450071,-1.964920,-1.640132,-1.299084,-1.573385,-0.780261,-1.314010,0.645328,-0.207860
3,an4,201904_ymaze_cfos,CNO_control_no_reversal,-0.132994,0.267429,4.023499,0.100342,-0.837821,-1.004073,0.484644,-0.456014,0.370308,0.876614,0.810062,1.157876
4,an5,201904_ymaze_cfos,CNO_control_no_reversal,0.880504,0.739237,-0.791744,-2.461296,0.574391,-0.652304,0.768235,-0.467711,-0.835604,0.000000,-1.469363,-1.127217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,dadult_pc_crus1_1,201810_adultacutePC_ymaze_cfos,vector_control_reversal,-1.004587,-0.105403,-0.362604,-0.931228,0.675822,0.000000,0.000000,-0.814602,-0.299724,1.702537,0.591808,1.464750
1,dadult_pc_crus1_3,201810_adultacutePC_ymaze_cfos,vector_control_reversal,0.004833,-1.135547,1.213210,0.000000,0.761164,1.407860,1.079762,-2.229711,-0.163084,-0.327450,-1.633412,1.413953
2,dadult_pc_crus1_4,201810_adultacutePC_ymaze_cfos,vector_control_reversal,0.000000,0.000000,0.000000,-0.723407,0.000000,0.595625,0.176835,0.000000,0.000000,1.892612,-1.633412,-0.841941
3,dadult_pc_lob6_14,201810_adultacutePC_ymaze_cfos,vector_control_reversal,1.163323,1.667290,2.409142,0.531314,-1.307560,-2.142305,-1.979964,0.166095,0.796048,-0.529826,0.757956,-1.090004


In [111]:
cdf.to_csv('../data/Jess_cfos_20201125_median_zscores_12subregions.csv')

## 122 structures

In [112]:
structures_df = pd.read_csv(structures_file)
structures_df

,name
0,Anterior amygdalar area
1,Basolateral amygdalar nucleus
2,Central amygdalar nucleus
3,Cortical amygdalar area
4,Intercalated amygdalar nucleus
...,...
117,Nucleus incertus
118,Pontine reticular nucleus
119,Nucleus raphe pontis
120,Subceruleus nucleus


In [113]:
structure_names = structures_df['name'] .values
structure_names

array(['Anterior amygdalar area', 'Basolateral amygdalar nucleus',
       'Central amygdalar nucleus', 'Cortical amygdalar area',
       'Intercalated amygdalar nucleus', 'Lateral amygdalar nucleus',
       'Posterior amygdalar nucleus', 'Somatomotor areas',
       'Somatosensory areas', 'Gustatory areas', 'Visceral area',
       'Auditory areas', 'Visual areas', 'Anterior cingulate area',
       'Prelimbic area', 'Infralimbic area', 'Orbital area',
       'Retrosplenial area', 'Posterior parietal association areas',
       'Temporal association areas', 'Perirhinal area', 'Ectorhinal area',
       'Hippocampal region', 'Retrohippocampal region',
       'Endopiriform nucleus', 'Caudoputamen', 'Nucleus accumbens',
       'Fundus of striatum', 'Lateral septal nucleus',
       'Septofimbrial nucleus', 'Septohippocampal nucleus', 'Pallidum',
       'Ventral anterior-lateral complex of the thalamus',
       'Ventral medial nucleus of the thalamus',
       'Ventral posterolateral nucleus of t

In [117]:
# Make a brain division dict from the list of regions
bdict_122 = {name:[name] for name in structure_names}

In [118]:
df_122 = make_all_division_df(bdict_122,count_type='frac')

In [119]:
df_list_122 = []
for batch in batches:
#     print(batch)
    control_conditions = batch_control_conditions_dict[batch]
    if len(control_conditions) < 1:
        continue
    print(batch)
    df_batch = df_122[df_122['batch'] == batch]
    df_batch_controls = df_batch[df_batch['condition'].isin(control_conditions)]
    medians=df_batch_controls.iloc[:,3:].median()
    mads=df_batch_controls.iloc[:,3:].mad()
    df_batch.iloc[:,3:] = (df_batch.iloc[:,3:]-medians)/mads
    df_list_122.append(df_batch)

201904_ymaze_cfos
201908_tpham_ymaze_cfos
202002_cfos
201810_adultacutePC_ymaze_cfos


In [120]:
cdf_122 = pd.concat(df_list_122)
cdf_122

,brain,batch,condition,Anterior amygdalar area,Basolateral amygdalar nucleus,Central amygdalar nucleus,Cortical amygdalar area,Intercalated amygdalar nucleus,Lateral amygdalar nucleus,Posterior amygdalar nucleus,...,Supragenual nucleus,Supratrigeminal nucleus,Tegmental reticular nucleus,Motor nucleus of trigeminal,Laterodorsal tegmental nucleus,Nucleus incertus,Pontine reticular nucleus,Nucleus raphe pontis,Subceruleus nucleus,Sublaterodorsal nucleus
0,an1,201904_ymaze_cfos,CNO_control_no_reversal,-0.681828,-0.997942,-0.543141,1.386560,-0.124473,-1.168820,2.100377,...,1.209734,-0.202191,-0.172074,0.417509,1.183191,0.494295,0.014956,0.000000,0.444195,0.980205
1,an2,201904_ymaze_cfos,CNO_control_no_reversal,3.146662,1.123205,-0.217072,2.065270,0.944997,1.230393,1.747160,...,0.630837,-0.026271,0.635518,0.922032,0.919216,0.745168,0.180435,-0.127043,1.125353,1.016236
2,an3,201904_ymaze_cfos,CNO_control_no_reversal,-0.486329,-1.362544,-1.719636,1.962787,-0.948464,-1.742749,2.475917,...,-0.395749,-0.458299,-0.054069,-0.802176,-2.147529,-1.187712,-0.669189,-1.068061,-0.500142,0.000000
3,an4,201904_ymaze_cfos,CNO_control_no_reversal,3.604553,1.312433,0.330573,-0.910128,3.406423,0.211098,0.529206,...,0.983335,-0.088146,5.130365,0.045092,-1.057936,0.460872,1.465549,2.067677,1.007645,1.665182
4,an5,201904_ymaze_cfos,CNO_control_no_reversal,2.865791,-1.284344,0.834181,0.563556,-0.072873,-1.154238,0.000000,...,-0.395749,-1.060526,-0.731436,-1.283310,-2.047593,-1.902590,-0.467747,-1.528463,-1.327061,-1.281046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,dadult_pc_crus1_1,201810_adultacutePC_ymaze_cfos,vector_control_reversal,0.000000,-0.634344,0.000000,0.000000,0.000000,1.084876,0.000000,...,NaN,0.000000,NaN,NaN,3.125000,NaN,1.578458,NaN,NaN,NaN
1,dadult_pc_crus1_3,201810_adultacutePC_ymaze_cfos,vector_control_reversal,0.730002,1.077664,1.462559,1.852391,0.716274,-0.123659,2.686082,...,NaN,0.000000,NaN,NaN,0.000000,NaN,1.364624,NaN,NaN,NaN
2,dadult_pc_crus1_4,201810_adultacutePC_ymaze_cfos,vector_control_reversal,0.914724,0.000000,-0.832316,-0.552699,-1.923897,0.000000,-0.735128,...,NaN,0.000000,NaN,NaN,0.000000,NaN,-0.877352,NaN,NaN,NaN
3,dadult_pc_lob6_14,201810_adultacutePC_ymaze_cfos,vector_control_reversal,-2.757429,2.150149,0.796095,0.672592,2.102963,2.073108,0.097781,...,NaN,0.000000,NaN,NaN,0.000000,NaN,-0.958025,NaN,NaN,NaN


In [121]:
cdf_122.to_csv('../data/Jess_cfos_20201125_median_zscores_122subregions.csv')